# Preliminary Analysis

### Data Cleaning Code
Code for cleaning and processing your data. Include a data dictionary for your transformed dataset.

- Data Dictionary for Air Quality
    - **indicator id:** id for each name
    - **name:** classify the sample in the air
    - **measure:** how the indicator is measured
    - **measure info:** information about the measure
    - **geo type name:** geography type, UHF stands for United Hospital Fund neighborhoods
    - **geo place name:** neighborhood name
    - **time period:** time frame
    - **start_date:** date started
    <br><br>
- Data Dictionary for Traffic Volume
    - **requestId:** unique id generated for each counts request
    - **boro:** lists which of the five diviions of New York City the location is within
    - **vol:** total sum of count collected within 15 minute increments
    - **segmentId:** The ID that idenifies each segment of a street
    - **wktgeom:** Geometry point of the location
    - **street:** street name of where traffic happened
    - **fromst:** start street of traffic
    - **tost:** end street where traffic volume was located
    - **direction:** text-based direction of traffic where the count took place
    - **date_time:** date at which it took place
    <br><br>
- Data Dictionary for 2020 mobility Dataset
    - **sub_region_2** which county it is
    - **date** date during recording
    - **retail_and_recreation_percent_change_from_baseline** mobility trends for places like restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.
    - **grocery_and_pharmacy_percent_change_from_baseline** mobility trends for places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies
    - **parks_percent_change_from_baseline** mobility trends for places like national parks, public beaches, marinas, dog parks, plazas, and public gardens
    - **transit_stations_percent_change_from_baseline** mobility trends for places like public transport hubs such as subway, bus, and train stations
    - **workplaces_percent_change_from_baseline** mobility trend for places of work
    - **residential_percent_change_from_baseline** mobility trends for places of residence
    
### Exploratory Analysis
Describe what work you have done so far and include the code. This may include descriptive statistics, graphs and charts, and preliminary models.

- We removed some columns that were irrelevant to what we want to predict as well as combine some columns that would fit together, such as the date and time.


### Challenges
Describe any challenges you've encountered so far. Let me know if there's anything you need help with!

- There were some challenges in figuring out what sort of data was necessary to include for our problem as it was targeted in New York City. 
- Figuring out the transformations to use on each dataset was also a challenge since there were many columns for each dataset and we had to find the ones that weren't relevant to our problem.
- There are some issues for the columns right now where there are some, such as segmentId in the Traffic Volume dataset where we are currently unsure if it's useful to keep or remove.
- Dealing with large datasets efficiently

### Future Work
Describe what work you are planning to complete for the final analysis.

- Future work includes using the cleaned data to use as inputs for models suited for classification such as Logisitc Regression and Linear Regression. 
- Make predictions using the models trained to obtain the accuracy scores to answer our questions
- Find the best model for accuracy as well as graph/chart the data to further understand it for future predictions.

### Contributions
Describe the contributions that each group member made.
- **Daniel Aguilar-Rodriguez**
    - Researched and acquired datasets
    - Helped present ideas during brainstorming session
    - Created jupyter notebook and helped clean datasets
    - Helped transform datasets and removed columns irrelevant to our work
    <br><br>
- **Jia Cong Lin**
    - Helped present ideas during brainstorming session
    - Helped define necessary columns for the mobility dataset
    - Assisted in determining columns to clean and define 
    <br><br>
- **Anvinh Truong**
    - Helped clean and define some columns for the datasets and dictionary
    - Helped present ideas during brainstorming session
    - Assisted in thinking of procedure to clean data columns

In [1]:
import pandas as pd
import numpy as np
import os
import requests

In [2]:
def download_data(csv_name):
    url_dict = {'air_quality': 'https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.csv', 
                'mobility_global': 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv', 
                'traffic_volume': 'https://data.cityofnewyork.us/api/views/7ym2-wayt/rows.csv'}
    
    response = requests.get(url_dict[csv_name])
    path = f'datasets/{csv_name}.csv'
    with open(path, 'wb') as f:
        f.write(response.content)

In [3]:
def csv_exists(csv_name):
    path = f'datasets/{csv_name}.csv'
    file_exists = os.path.exists(path)
    return file_exists

In [4]:
def create_df(csv_name):
    if not csv_exists(csv_name):
        download_data(csv_name)
    path = f'datasets/{csv_name}.csv'
    df = pd.read_csv(path)
    return df

In [5]:
def mkdir_if_not_exist():
    directory = 'datasets'
    if not os.path.exists(f'{directory}/'):
        os.mkdir(directory)

In [6]:
def create_all_df(csv_names):
    mkdir_if_not_exist()
    df_list = []
    
    for csv_name in csv_names:
        print(f'Creating {csv_name} df')
        df = create_df(csv_name)
        df_list.append(df)
        
    return df_list

In [7]:
csv_names = ['air_quality', 'mobility_global', 'traffic_volume']

air_quality, mobility_global, traffic_volume = create_all_df(csv_names)

Creating air_quality df
Creating mobility_global df


C:\Users\danie\AppData\Local\Temp\ipykernel_12908\89086087.py:5: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Creating traffic_volume df


## Air Quality Dataset Cleaning

In [8]:
print(air_quality.isnull().sum() / len(air_quality))

Unique ID         0.0
Indicator ID      0.0
Name              0.0
Measure           0.0
Measure Info      0.0
Geo Type Name     0.0
Geo Join ID       0.0
Geo Place Name    0.0
Time Period       0.0
Start_Date        0.0
Data Value        0.0
Message           1.0
dtype: float64


In [9]:
air_quality = air_quality.drop(['Message'], axis=1)
print(air_quality.isnull().sum() / len(air_quality))

Unique ID         0.0
Indicator ID      0.0
Name              0.0
Measure           0.0
Measure Info      0.0
Geo Type Name     0.0
Geo Join ID       0.0
Geo Place Name    0.0
Time Period       0.0
Start_Date        0.0
Data Value        0.0
dtype: float64


In [10]:
print(air_quality.nunique() / len(air_quality))

Unique ID         1.000000
Indicator ID      0.001365
Name              0.001179
Measure           0.000496
Measure Info      0.000496
Geo Type Name     0.000310
Geo Join ID       0.004466
Geo Place Name    0.007071
Time Period       0.002791
Start_Date        0.002233
Data Value        0.253443
dtype: float64


In [11]:
air_quality = air_quality.drop(['Unique ID'], axis=1)
print(air_quality.shape)
print(air_quality.nunique() / len(air_quality))

(16122, 10)
Indicator ID      0.001365
Name              0.001179
Measure           0.000496
Measure Info      0.000496
Geo Type Name     0.000310
Geo Join ID       0.004466
Geo Place Name    0.007071
Time Period       0.002791
Start_Date        0.002233
Data Value        0.253443
dtype: float64


In [12]:
air_quality = air_quality.drop(['Geo Join ID'], axis=1)
print(air_quality.shape)
print(air_quality.nunique() / len(air_quality))

(16122, 9)
Indicator ID      0.001365
Name              0.001179
Measure           0.000496
Measure Info      0.000496
Geo Type Name     0.000310
Geo Place Name    0.007071
Time Period       0.002791
Start_Date        0.002233
Data Value        0.253443
dtype: float64


In [13]:
air_quality.dtypes

Indicator ID        int64
Name               object
Measure            object
Measure Info       object
Geo Type Name      object
Geo Place Name     object
Time Period        object
Start_Date         object
Data Value        float64
dtype: object

In [14]:
air_quality.nunique()

Indicator ID        22
Name                19
Measure              8
Measure Info         8
Geo Type Name        5
Geo Place Name     114
Time Period         45
Start_Date          36
Data Value        4086
dtype: int64

In [15]:
air_quality['Time Period'].unique()

array(['Summer 2013', 'Summer 2014', 'Winter 2008-09', 'Summer 2009',
       'Summer 2010', 'Summer 2011', 'Summer 2012', 'Winter 2009-10',
       '2005-2007', '2013', '2005', '2009-2011', 'Winter 2010-11',
       'Winter 2011-12', 'Winter 2012-13', 'Annual Average 2009',
       'Annual Average 2010', 'Annual Average 2011',
       'Annual Average 2012', 'Annual Average 2013', '2015',
       'Winter 2013-14', 'Annual Average 2014', '2011', 'Winter 2014-15',
       '2016', 'Annual Average 2015', 'Summer 2015', 'Winter 2015-16',
       'Summer 2016', 'Annual Average 2016', 'Summer 2017', '2012-2014',
       'Summer 2018', 'Annual Average 2017', 'Summer 2019',
       'Winter 2016-17', 'Annual Average 2018', 'Winter 2017-18',
       '2015-2017', 'Summer 2020', 'Annual Average 2019',
       'Winter 2018-19', 'Annual Average 2020', 'Winter 2019-20'],
      dtype=object)

In [16]:
air_quality['Time Period'].value_counts() / len(air_quality)

2012-2014              0.029773
2005-2007              0.029773
2009-2011              0.029773
2015-2017              0.029773
Summer 2018            0.026237
Summer 2017            0.026237
Summer 2016            0.026237
Winter 2015-16         0.026237
Summer 2015            0.026237
Summer 2019            0.026237
Winter 2014-15         0.026237
Summer 2014            0.026237
Winter 2013-14         0.026237
Summer 2020            0.026237
Summer 2013            0.026237
Winter 2009-10         0.026237
Summer 2011            0.026237
Winter 2011-12         0.026237
Winter 2010-11         0.026237
Winter 2008-09         0.026237
Summer 2009            0.026237
Summer 2010            0.026237
Winter 2012-13         0.026237
Summer 2012            0.026237
2005                   0.025245
2016                   0.019911
Annual Average 2017    0.017492
Winter 2018-19         0.017492
Winter 2016-17         0.017492
Annual Average 2020    0.017492
Annual Average 2018    0.017492
Winter 2

In [17]:
air_quality['Start_Date'].unique()

array(['06/01/2013', '06/01/2014', '12/01/2008', '06/01/2009',
       '06/01/2010', '06/01/2011', '06/01/2012', '12/01/2009',
       '01/01/2005', '01/01/2013', '01/01/2009', '12/01/2010',
       '12/01/2011', '12/01/2012', '01/01/2015', '12/01/2013',
       '01/01/2011', '12/01/2014', '01/01/2016', '06/01/2015',
       '12/01/2015', '05/31/2016', '12/31/2015', '06/01/2017',
       '01/02/2012', '06/01/2018', '01/01/2017', '06/01/2019',
       '12/01/2016', '01/01/2018', '12/01/2017', '06/01/2020',
       '01/01/2019', '12/01/2018', '01/01/2020', '12/01/2019'],
      dtype=object)

In [18]:
air_quality['Start_Date'] = pd.to_datetime(air_quality['Start_Date'], infer_datetime_format=True)

In [19]:
air_quality['Start_Date'].min()

Timestamp('2005-01-01 00:00:00')

In [20]:
air_quality['Start_Date'].value_counts().sort_index() / len(air_quality)

2005-01-01    0.055018
2008-12-01    0.043729
2009-01-01    0.029773
2009-06-01    0.026237
2009-12-01    0.043729
2010-06-01    0.026237
2010-12-01    0.043729
2011-01-01    0.013274
2011-06-01    0.026237
2011-12-01    0.043729
2012-01-02    0.029773
2012-06-01    0.026237
2012-12-01    0.043729
2013-01-01    0.008932
2013-06-01    0.026237
2013-12-01    0.043729
2014-06-01    0.026237
2014-12-01    0.026237
2015-01-01    0.056197
2015-06-01    0.026237
2015-12-01    0.026237
2015-12-31    0.017492
2016-01-01    0.019911
2016-05-31    0.026237
2016-12-01    0.017492
2017-01-01    0.017492
2017-06-01    0.026237
2017-12-01    0.017492
2018-01-01    0.017492
2018-06-01    0.026237
2018-12-01    0.017492
2019-01-01    0.017492
2019-06-01    0.026237
2019-12-01    0.017492
2020-01-01    0.017492
2020-06-01    0.026237
Name: Start_Date, dtype: float64

## Traffic Volume Dataset Cleaning

In [21]:
traffic_volume.sample(10)

,RequestID,Boro,Yr,M,D,HH,MM,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction
11685516,11992,Brooklyn,2013,3,3,7,0,15,16879,POINT (975115.7 162867.7),4 AV,99 ST,MARINE AV,NB
325369,17293,Queens,2014,6,14,12,30,214,129667,POINT (1003731.2 218927),21 ST,31 AV,30 DR,SB
20333570,29461,Manhattan,2019,1,14,6,45,121,257392,POINT (990678.0274413553 213162.08271735016),EAST 42 STREET,Flushing Line,Flushing Line,WB
24230971,17780,Brooklyn,2014,8,10,16,30,137,25699,POINT (995287.9 151809.5),GUIDER AV,SHORE PKWY/BANNER AV,CONEY ISLAND AV,EB
19742589,8282,Manhattan,2012,6,17,19,45,85,38635,POINT (997699.6 234324.3),FRED DOUGLASS BLVD,W 124 ST,W 125 ST,NB
14224913,11880,Queens,2013,6,12,17,0,282,84439,POINT (1029709.8 219883.6),WHE SB EN ULMER ST,WHITESTONE EXWY,WHITESTONE EXPRESSWAY SR W/ULMER ST,SB
4553119,21967,Queens,2015,11,5,22,0,10,131989,POINT (1048994.7511838316 215768.91792601327),46 AVENUE,Alley,215 Place,EB
16600547,29131,Bronx,2018,10,15,0,30,15,78301,POINT (1011773.2275664982 242140.90300823937),EAST 169 STREET,Boston Road,Boston Road,WB
7415160,17398,Manhattan,2014,6,7,10,0,173,38235,POINT (994194.3 231714.2),AMSTERDAM AV,W 109 ST,CATHEDRAL PKWY,NB
6480495,27162,Brooklyn,2017,12,8,11,0,75,28215,POINT (994349.0578452068 175811.22650137288),CHURCH AVENUE,East 17 Street,Brighton Line,WB


In [22]:
traffic_volume.shape

(27190511, 14)

In [23]:
print(traffic_volume.isnull().sum() / len(traffic_volume))

RequestID    0.000000
Boro         0.000000
Yr           0.000000
M            0.000000
D            0.000000
HH           0.000000
MM           0.000000
Vol          0.000000
SegmentID    0.000000
WktGeom      0.000000
street       0.000000
fromSt       0.000000
toSt         0.000074
Direction    0.000000
dtype: float64


In [24]:
print(traffic_volume.nunique() / len(traffic_volume))

RequestID    2.607527e-04
Boro         1.838877e-07
Yr           5.884406e-07
M            4.413304e-07
D            1.140104e-06
HH           8.826609e-07
MM           1.471101e-07
Vol          1.476986e-04
SegmentID    5.499345e-04
WktGeom      7.525787e-04
street       2.482484e-04
fromSt       2.361486e-04
toSt         2.175391e-04
Direction    2.206652e-07
dtype: float64


In [25]:
traffic_volume.nunique()

RequestID     7090
Boro             5
Yr              16
M               12
D               31
HH              24
MM               4
Vol           4016
SegmentID    14953
WktGeom      20463
street        6750
fromSt        6421
toSt          5915
Direction        6
dtype: int64

In [26]:
traffic_volume.dtypes

RequestID     int64
Boro         object
Yr            int64
M             int64
D             int64
HH            int64
MM            int64
Vol           int64
SegmentID     int64
WktGeom      object
street       object
fromSt       object
toSt         object
Direction    object
dtype: object

In [27]:
traffic_volume.Yr.min()

2000

In [28]:
traffic_volume = traffic_volume[traffic_volume['Yr'] >= 2005]

In [29]:
traffic_volume.shape

(27188607, 14)

In [30]:
traffic_volume['Yr'].value_counts().sort_index()

2006        664
2007      11780
2008      68591
2009    1012766
2010    1421397
2011    1238391
2012    2434583
2013    2829656
2014    3708367
2015    3232005
2016    3362243
2017    3013530
2018    2046443
2019    2365633
2020     442558
Name: Yr, dtype: int64

In [31]:
traffic_volume = traffic_volume[traffic_volume['Yr'] > 2008]

In [32]:
traffic_volume.shape

(27107572, 14)

In [33]:
traffic_volume['date_time'] = pd.to_datetime(dict(year=traffic_volume.Yr, \
                                                  month=traffic_volume.M, \
                                                  day=traffic_volume.D, \
                                                  hour=traffic_volume.HH, \
                                                  minute=traffic_volume.MM))

In [34]:
traffic_volume = traffic_volume.drop(['Yr', 'M', 'D', 'HH', 'MM'], axis=1)

In [35]:
traffic_volume.sample(10)

,RequestID,Boro,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,date_time
15274714,11066,Brooklyn,10,108099,POINT (989523.2 155904.6),W 10 ST,86 ST,LAKE PL,EB,2013-06-05 05:00:00
21473004,30153,Queens,1116,137935,POINT (1063242.807147359 216251.8599977011),GRAND CENTRAL PARKWAY,Dead End,Dead end,WB,2019-05-10 13:15:00
4832246,26756,Brooklyn,150,265958,POINT (983819.7489572052 177375.13403359734),5 AVENUE,34 Street,36 Street,NB,2017-10-19 11:00:00
149591,31075,Staten Island,143,12946,POINT (955693.0548479398 161255.60603713675),OCEAN TERRACE,Dead End,Milford Drive,SB,2019-09-25 17:00:00
3729440,12219,Brooklyn,154,23979,POINT (986331.8 190452.8),COURT ST APPROACH TO PACIFIC ST,PACIFIC ST,ATLANTIC AV,SB,2011-11-11 17:30:00
6002827,29350,Brooklyn,24,50935,POINT (1022772.5499493582 182175.90161311213),FORBELL STREET,Loring Avenue,Stanley Avenue,NB,2018-11-18 09:30:00
18068472,19913,Queens,189,68030,POINT (1007471.7563886127 219943.302069417),ASTORIA BOULEVARD NORTH,33 Street,32 Street,WB,2014-10-30 12:45:00
108429,32417,Queens,81,101621,POINT (1055667.922729934 216597.78334720692),DOUGLASTON PARKWAY,Maryland Road,Van Zandt Avenue,SB,2020-11-14 15:30:00
10817944,17722,Manhattan,116,71072,POINT (1002137 246051.7),AMSTERDAM AV,W 171 ST/EDGECOMBE AV,W 172 ST,SB,2014-07-30 15:15:00
3760403,19648,Queens,104,184079,POINT (1063632.9629087522 216037.56884304955),GRAND CENTRAL PARKWAY SR SOUTH,Dead End,North Shore Tower Driveway,WB,2015-06-05 12:45:00


In [36]:
traffic_volume['date_time'].dt.year.value_counts().sort_index()

2009    1012766
2010    1421397
2011    1238391
2012    2434583
2013    2829656
2014    3708367
2015    3232005
2016    3362243
2017    3013530
2018    2046443
2019    2365633
2020     442558
Name: date_time, dtype: int64

## Mobility dataset cleaning

In [37]:
mobility_global.head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
country_region_code,AE,AE,AE,AE,AE,AE,AE,AE,AE,AE
country_region,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates
sub_region_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_region_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
metro_area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iso_3166_2_code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
census_fips_code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
place_id,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM
date,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24
retail_and_recreation_percent_change_from_baseline,0.0,1.0,-1.0,-2.0,-2.0,-2.0,-3.0,-2.0,-1.0,-3.0


In [38]:
mobility_global.shape

(11730025, 15)

In [39]:
mobility_nyc = mobility_global[mobility_global['country_region_code'].eq('US') & 
                               mobility_global['sub_region_1'].eq('New York') & 
                               mobility_global['sub_region_2'].str.contains('Bronx|Kings|New York|Queens|Richmond')]

In [40]:
mobility_nyc.sample(10).transpose()

,10589023,10551194,10589424,10570809,10571233,10570688,10570892,10587467,10551636,10577741
country_region_code,US,US,US,US,US,US,US,US,US,US
country_region,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States
sub_region_1,New York,New York,New York,New York,New York,New York,New York,New York,New York,New York
sub_region_2,Richmond County,Bronx County,Richmond County,Kings County,Kings County,Kings County,Kings County,Queens County,Bronx County,New York County
metro_area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iso_3166_2_code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
census_fips_code,36085.0,36005.0,36085.0,36047.0,36047.0,36047.0,36047.0,36081.0,36005.0,36061.0
place_id,ChIJOwE7_GTtwokR1V_vES61lRI,ChIJBUEf6ovgwokRwlazSIxIpsk,ChIJOwE7_GTtwokR1V_vES61lRI,ChIJOwE7_GTtwokRs75rhW4_I6M,ChIJOwE7_GTtwokRs75rhW4_I6M,ChIJOwE7_GTtwokRs75rhW4_I6M,ChIJOwE7_GTtwokRs75rhW4_I6M,ChIJgav5pFbxwokRno6Tc5x2GL8,ChIJBUEf6ovgwokRwlazSIxIpsk,ChIJOwE7_GTtwokRFq0uOwLSE9g
date,2020-04-17,2020-04-25,2021-05-23,2020-12-07,2022-02-04,2020-08-08,2021-02-28,2021-05-14,2021-07-11,2021-05-13
retail_and_recreation_percent_change_from_baseline,-62.0,-55.0,-11.0,-31.0,-36.0,-25.0,-38.0,-14.0,-19.0,-47.0


In [41]:
mobility_nyc.shape

(4870, 15)

In [42]:
mobility_nyc.isnull().sum() / len(mobility_nyc)

country_region_code                                   0.000000
country_region                                        0.000000
sub_region_1                                          0.000000
sub_region_2                                          0.000000
metro_area                                            1.000000
iso_3166_2_code                                       1.000000
census_fips_code                                      0.000000
place_id                                              0.000000
date                                                  0.000000
retail_and_recreation_percent_change_from_baseline    0.000000
grocery_and_pharmacy_percent_change_from_baseline     0.000000
parks_percent_change_from_baseline                    0.001643
transit_stations_percent_change_from_baseline         0.000000
workplaces_percent_change_from_baseline               0.000000
residential_percent_change_from_baseline              0.000000
dtype: float64

In [43]:
mobility_nyc = mobility_nyc.drop(['metro_area', 'iso_3166_2_code'], axis=1)

In [44]:
mobility_nyc.isnull().sum()

country_region_code                                   0
country_region                                        0
sub_region_1                                          0
sub_region_2                                          0
census_fips_code                                      0
place_id                                              0
date                                                  0
retail_and_recreation_percent_change_from_baseline    0
grocery_and_pharmacy_percent_change_from_baseline     0
parks_percent_change_from_baseline                    8
transit_stations_percent_change_from_baseline         0
workplaces_percent_change_from_baseline               0
residential_percent_change_from_baseline              0
dtype: int64

In [45]:
mobility_nyc = mobility_nyc.drop(['country_region_code', 'country_region', 'sub_region_1'], axis=1)

In [46]:
mobility_nyc.sample(10)

,sub_region_2,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
10587336,Queens County,36081.0,ChIJgav5pFbxwokRno6Tc5x2GL8,2021-01-03,-48.0,-26.0,-38.0,-51.0,-26.0,11.0
10589916,Richmond County,36085.0,ChIJOwE7_GTtwokR1V_vES61lRI,2022-09-27,-13.0,-8.0,-13.0,-31.0,-39.0,7.0
10571324,Kings County,36047.0,ChIJOwE7_GTtwokRs75rhW4_I6M,2022-05-06,-31.0,-19.0,-38.0,-37.0,-30.0,8.0
10578001,New York County,36061.0,ChIJOwE7_GTtwokRFq0uOwLSE9g,2022-01-28,-50.0,-15.0,-45.0,-44.0,-46.0,15.0
10587796,Queens County,36081.0,ChIJgav5pFbxwokRno6Tc5x2GL8,2022-04-08,-16.0,1.0,41.0,-23.0,-21.0,5.0
10588987,Richmond County,36085.0,ChIJOwE7_GTtwokR1V_vES61lRI,2020-03-12,3.0,24.0,1.0,-9.0,-3.0,3.0
10570967,Kings County,36047.0,ChIJOwE7_GTtwokRs75rhW4_I6M,2021-05-14,-20.0,-5.0,62.0,-36.0,-42.0,10.0
10570617,Kings County,36047.0,ChIJOwE7_GTtwokRs75rhW4_I6M,2020-05-29,-47.0,-14.0,31.0,-55.0,-58.0,25.0
10587597,Queens County,36081.0,ChIJgav5pFbxwokRno6Tc5x2GL8,2021-09-21,-6.0,8.0,74.0,-25.0,-35.0,7.0
10571411,Kings County,36047.0,ChIJOwE7_GTtwokRs75rhW4_I6M,2022-08-01,-27.0,-14.0,2.0,-38.0,-39.0,8.0


In [47]:
mobility_nyc.groupby(['sub_region_2', 'census_fips_code'])['place_id'].nunique()

sub_region_2     census_fips_code
Bronx County     36005.0             1
Kings County     36047.0             1
New York County  36061.0             1
Queens County    36081.0             1
Richmond County  36085.0             1
Name: place_id, dtype: int64

In [48]:
mobility_nyc = mobility_nyc.drop(['census_fips_code', 'place_id'], axis=1)

In [49]:
mobility_nyc.sample(10).transpose()

,10571119,10570555,10589058,10589784,10577561,10570890,10587212,10578176,10589451,10571354
sub_region_2,Kings County,Kings County,Richmond County,Richmond County,New York County,Kings County,Queens County,New York County,Richmond County,Kings County
date,2021-10-13,2020-03-28,2020-05-22,2022-05-18,2020-11-14,2021-02-26,2020-09-01,2022-07-22,2021-06-19,2022-06-05
retail_and_recreation_percent_change_from_baseline,-17.0,-67.0,-47.0,-8.0,-54.0,-35.0,-20.0,-40.0,-14.0,-22.0
grocery_and_pharmacy_percent_change_from_baseline,-2.0,-30.0,-9.0,-3.0,-18.0,-12.0,0.0,-27.0,5.0,-20.0
parks_percent_change_from_baseline,42.0,-51.0,-5.0,-1.0,-23.0,-15.0,97.0,-6.0,6.0,84.0
transit_stations_percent_change_from_baseline,-28.0,-62.0,-56.0,-29.0,-46.0,-45.0,-48.0,-36.0,-20.0,-29.0
workplaces_percent_change_from_baseline,-32.0,-49.0,-52.0,-21.0,-21.0,-46.0,-44.0,-49.0,-15.0,-29.0
residential_percent_change_from_baseline,7.0,19.0,23.0,4.0,6.0,15.0,14.0,9.0,-1.0,0.0


In [50]:
mobility_nyc.shape

(4870, 8)

In [51]:
mobility_nyc.dtypes

sub_region_2                                           object
date                                                   object
retail_and_recreation_percent_change_from_baseline    float64
grocery_and_pharmacy_percent_change_from_baseline     float64
parks_percent_change_from_baseline                    float64
transit_stations_percent_change_from_baseline         float64
workplaces_percent_change_from_baseline               float64
residential_percent_change_from_baseline              float64
dtype: object

## Transformed Datasets

In [52]:
traffic_volume.head(15)

,RequestID,Boro,Vol,SegmentID,WktGeom,street,fromSt,toSt,Direction,date_time
0,20856,Queens,9,171896,POINT (1052296.600156678 199785.26932711253),94 AVENUE,207 Street,Francis Lewis Boulevard,WB,2015-06-23 23:30:00
1,21231,Staten Island,6,9896,POINT (942668.0589509147 171441.21296926),RICHMOND TERRACE,Wright Avenue,Emeric Court,WB,2015-09-14 04:15:00
2,29279,Bronx,85,77817,POINT (1016508.0034050211 235221.59092266942),HUNTS POINT AVENUE,Whittier Street,Randall Avenue,NB,2017-10-19 04:30:00
3,27019,Brooklyn,168,188023,POINT (992925.4316054962 184116.82855457635),FLATBUSH AVENUE,Brighton Line,Brighton Line,NB,2017-11-07 18:30:00
4,26734,Manhattan,355,137516,POINT (1004175.9505178436 247779.63624949602),WASHINGTON BRIDGE,Harlem River Shoreline,Harlem River Shoreline,EB,2017-11-03 22:00:00
5,26015,Bronx,11,86053,POINT (1021709.470909429 248612.86356908735),WALLACE AVENUE,Rhinelander Avenue,Bronxdale Avenue,NB,2017-06-17 01:45:00
6,2033,Manhattan,99,70683,POINT (1000954.8 243914.9),S/B AMSTERDAM AVE @ W 162 ST,ST NICHOLAS AV/W 162 ST,W 163 ST,SB,2009-09-01 18:30:00
7,23133,Queens,232,101101,POINT (1050277.3347521287 216784.58047417598),NORTHERN BOULEVARD,220 Place,220 Street,WB,2016-03-21 09:45:00
8,32417,Queens,18,147877,POINT (1044172.6626552071 200130.04842303603),MIDLAND PARKWAY,Dalny Road,Connector,SB,2020-11-14 02:15:00
9,26198,Bronx,2,85935,POINT (1021747.2311522859 242463.04655740186),THIERIOT AVENUE,Gleason Avenue,Pelham Line,NB,2017-06-22 04:30:00


In [53]:
air_quality.sample(10)

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value
11292,659,O3-Attributable Asthma Emergency Department Vi...,Estimated Annual Rate- 18 Yrs and Older,"per 100,000 adults",UHF42,Upper East Side,2012-2014,2012-01-02,8.80
10405,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,UHF34,Bensonhurst - Bay Ridge,Summer 2016,2016-05-31,7.94
2235,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Tottenville and Great Kills (CD3),Summer 2010,2010-06-01,9.42
14077,661,O3-Attributable Asthma Hospitalizations,Estimated Annual Rate- 18 Yrs and Older,"per 100,000 adults",UHF42,Hunts Point - Mott Haven,2015-2017,2015-01-01,14.20
1871,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Hillcrest and Fresh Meadows (CD8),Winter 2008-09,2008-12-01,27.99
12276,375,Nitrogen Dioxide (NO2),Mean,ppb,Borough,Staten Island,Annual Average 2017,2017-01-01,13.37
5996,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,UHF42,West Queens,Summer 2012,2012-06-01,10.77
9746,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Brownsville (CD16),Winter 2014-15,2014-12-01,25.85
7415,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,Rego Park and Forest Hills (CD6),Annual Average 2014,2013-12-01,20.88
7008,375,Nitrogen Dioxide (NO2),Mean,ppb,UHF42,West Queens,Summer 2014,2014-06-01,16.64


In [54]:
mobility_nyc.sample(10)

,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
10587578,Queens County,2021-09-02,-3.0,8.0,109.0,-30.0,-43.0,11.0
10577913,New York County,2021-11-01,-34.0,-12.0,-11.0,-33.0,-44.0,11.0
10571159,Kings County,2021-11-22,-18.0,-3.0,-3.0,-27.0,-30.0,8.0
10577366,New York County,2020-05-03,-82.0,-38.0,-36.0,-74.0,-47.0,14.0
10587711,Queens County,2022-01-13,-20.0,4.0,11.0,-34.0,-29.0,11.0
10578206,New York County,2022-08-21,-33.0,-24.0,14.0,-20.0,-20.0,1.0
10551788,Bronx County,2021-12-10,-23.0,-4.0,-21.0,-26.0,-26.0,6.0
10587787,Queens County,2022-03-30,-21.0,-3.0,4.0,-26.0,-21.0,6.0
10589638,Richmond County,2021-12-23,10.0,35.0,-24.0,-28.0,-32.0,9.0
10589079,Richmond County,2020-06-12,-41.0,-2.0,38.0,-48.0,-45.0,18.0
